<a id="title_ID"></a>
# JWST Pipeline Validation Notebook: 
# < pipeline name >, < step name>

<span style="color:red"> **Instruments Affected**</span>: e.g., FGS, MIRI, NIRCam, NIRISS, NIRSpec 

### Table of Contents
Follow this general outline. Additional sections may be added and others can be excluded, as needed. Sections in with a (\*) symbol are required.

<div style="text-align: left"> 
    
<br> [Introduction\*](#intro)
<br> [JWST CalWG Algorithm\*](#algorithm)
<br> [Defining Terms](#terms)
<br> [Test Description\*](#description)
<br> [Data Description\*](#data_descr)
<br> [Imports\*](#imports)
<br> [Loading the Data\*](#data_load)
<br> [Run the Pipeline](#pipeline)
<br> [Perform Tests or Visualization](#testing) 
<br> [About This Notebook\*](#about)
<br>    

</div>

<a id="intro"></a>
# Introduction

Give a short introduction explaining the purpose of this notebook and providing relevant links. Here is an example: 

> This is the validation notebook for the source catalog step. This step creates a final catalog of source photometry and morphologies. Here we are testing the step for accuracy of the catalog. For more information on the pipeline step visit the links below. 

> Step description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/source_catalog/index.html

> Pipeline code: https://github.com/spacetelescope/jwst/tree/master/jwst/source_catalog

[Top of Page](#title_ID)

<a id="algorithm"></a>
# JWST CalWG Algorithm

Provide a short description of the implemented algorithm, and/or link to the Confluence page.

For example: 
> https://outerspace.stsci.edu/display/JWSTCC/Vanilla+Point+Source+Catalog 


[Top of Page](#title_ID)

<a id="terms"></a>
# Defining Terms

If necessary, provide terms or acronymns that may not be known a general audience (i.e., a new team member or an external user). For example:

> JWST: James Webb Space Telescope

> NIRSpec: Near-Infrared Spectrogragh


[Top of Page](#title_ID)

<a id="description"></a>
# Test Description
Provide a description of the test that is being performed.

For example:

>This test is performed by creating a set of simulated data with multiple point sources located at specified coordinates. The simulator puts in the expected distortion, so the initial output data comes out of the simulator in distorted coordinates. When this data is then run through calwebb_detector1, calwebb_image2 and calwebbb_image3, the combined, undistorted image should have the point sources registered at the expected locations. In flight, this test can be repeated with known stars that should be found at their expected coordinates.


[Top of Page](#title_ID)

<a id="data_descr"></a>
# Data Description
Provide a description of the test data that is being used.

For example:

>The set of data used in this particular test were created with the MIRI Data Simulator (MIRISim). The simulator created four imaging mode files, two exposures each at two different dither positions, using the specified filter.


[Top of Page](#title_ID)

<a id="tempdir"></a>
# Set up Temporary Directory
The following cell sets up a temporary directory (using python's `tempfile.TemporaryDirectory()`), and changes the script's active directory into that directory (using python's `os.chdir()`). This is so that, when the notebook is run through, it will download files to (and create output files in) the temporary directory rather than in the notebook's directory. This makes cleanup significantly easier (since all output files are deleted when the notebook is shut down), and also means that different notebooks in the same directory won't interfere with each other when run by the automated webpage generation process.

If you want the notebook to generate output in the notebook's directory, simply don't run this cell.

If you have a file (or files) that are kept in the notebook's directory, and that the notebook needs to use while running, you can copy that file into the directory (the code to do so is present below, but commented out).

[Top of Page](#title_ID)

In [1]:
# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
import shutil
data_dir = TemporaryDirectory()

# If you have files that are in the notebook's directory, but that the notebook will need to use while
# running, copy them into the temporary directory here.
#
# files = ['name_of_file']
# for file_name in files:
#     shutil.copy(file_name, os.path.join(data_dir.name, file_name))

os.chdir(data_dir.name)

<a id="imports"></a>
# Imports
List the package imports and why they are relevant to this notebook.


* astropy.io for opening fits files
* inspect to get the docstring of our objects.
* IPython.display for printing markdown output
* jwst.datamodels for building model for JWST Pipeline
* jwst.module.PipelineStep is the pipeline step being tested
* matplotlib.pyplot.plt to generate plot


[Top of Page](#title_ID)

In [2]:
# import inspect
# from astropy.io import fits
# from IPython.display import Markdown
# from jwst.datamodels import RampModel
# from jwst.module import PipelineStep
# import matplotlib.pyplot as plt

<a id="data_load"></a>
# Loading the Data

### Data for internal use: Artifactory method
Artifactory should be used for data that is for internal use only.

1. Create a [Jira "Task" Issue in the JWST Simulations Jira project](https://jira.stsci.edu/issues/?jql=project%20%3D%20JWSTSIMS%20AND%20resolution%20%3D%20Unresolved%20ORDER%20BY%20priority%20DESC%2C%20updated%20DESC) requesting to have your data added to Artifactory. Assign the ticket to Misty Cracraft ([@cracraft](https://github.com/cracraft)) or Alicia Canipe ([@aliciacanipe](https://github.com/aliciacanipe)), and provide more information about the data: simulation information, data location, and pipeline step(s). Once your data has been added to Artifactory, Misty Cracraft ([@cracraft](https://github.com/cracraft)) or Alicia Canipe ([@aliciacanipe](https://github.com/aliciacanipe)) will resolve the issue and notify you that your data is ready to be used (the full path to the data will be provided by the person who notified you that your data was ingested successfully).  

2. Make sure you have the proper OS environmental variable set to access STScI's instance of Artifactory. This can be done via command line or put into a setup file like a ```.bash_profile``` file.

```
export TEST_BIGDATA=https://bytesalad.stsci.edu/artifactory/
```

3. Make sure your environment has ```ci_watson``` installed.
```
pip install ci_watson
```

4. In your notebook, import the ```ci_watson``` package needed.

```python
from ci_watson.artifactory_helpers import get_bigdata
```

5. Read in each file stored in Artifactory (the full path should have been provided by the person who ingested the data).

```python
satfile = get_bigdata('jwst_validation_notebooks',
                                     'validation_data',
                                     'jump',
                                    'jump_miri_test',
                                    'miri_sat_55k.fits')
```

### Data for external use: Box method
Artifactory is only accessible to internal users on the STScI network. If you would like to contribute a test notebook that uses externally available data, this test data should be stored in a Box folder instead. The final workflow using Box is still in discussion, but for now you can use a Box folder with the correct permissions set up:

```python
from astropy.utils.data import download_file

main_box_url ="https://data.science.stsci.edu/redirect/JWST/TSO/pipeline_testing_miri_ima_tso/"
filename = 'pipetest_miri_imtso_FULL_10g10i_F770W.fits'
file = download_file(mainurl+filename)
```

Box assigns a default alpha-numerical string as the filename, so you may want to update the filename before processing, or verify that the format is correct. Depending on the data, you can try:

```python
# open file into correct format and write to local disk for processing
hdu = fits.open(file)
hdu.info()
hdu.writeto(filename)
```
or use a ```jwst datamodel```:

```python
from jwst.datamodels import RampModel
model = RampModel(file)
model.save(filename)
```

[Top of Page](#title_ID)

<a id="pipeline"></a>
# Run the Steps or Pipeline

Run the steps or the pipeline whose outputs will be validated for this test. For example:

```python
# import the pipeline you want to run (e.g., ramps-to-slopes)
from jwst.pipeline import calwebb_detector1

# initialize
m = calwebb_detector1.Detector1Pipeline()

# make changes to the parameters/reference files used
m.saturation.override_saturation = 'mysatfile.fits'
m.superbias.override_superbias = 'mysuperbias.fits'
m.refpix.odd_even_rows = False

# skip steps you don't want to run
m.group_scale.skip = True
m.ipc.skip = True
m.dark_current.skip = True
m.persistence.skip = True

# name your output file
m.output_file = 'myoutputfilename.fits'

# run the pipeline with these paramters
m.run('uncalfile.fits')
```  

[Top of Page](#title_ID)

<a id="testing"></a>
# Perform Tests or Visualization

Perform the validation tests described previously. Generate plots, tables, diagnostics, etc. 

[Top of Page](#title_ID)

<a id="about_ID"></a>
## About this Notebook
**Author:** Author Name, Job Title, Branch Name
<br>**Updated On:** MM/DD/YYYY

[Top of Page](#title_ID)
<img style="float: right;" src="./stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="stsci_pri_combo_mark_horizonal_white_bkgd" width="200px"/> 